## Preparando o Ambiente

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


pio.renderers.default = "notebook_connected"

# Carregar dados
df = pd.read_csv("../data/relacao-pessoas-limpo.csv", parse_dates=["data_vacinacao"])
df.head()


,_id,sexo,grupo,vacina,lote,dose,data_vacinacao,local_vacinacao,ano,mes,dia,dose_tipo
0,1,MASCULINO,TRABALHADORES INDUSTRIAIS E BANCÁRIOS,2 - CHADOX1NCOV-19 - OXFORD/ASTRAZENECA (FIOCRUZ),216VCD206Z,1,2021-07-26,DRIVE THRU PARQUE DE EXPOSIÇÃO,2021,7,26,1ª DOSE
1,2,MASCULINO,TRABALHADORES INDUSTRIAIS E BANCÁRIOS,2 - CHADOX1NCOV-19 - OXFORD/ASTRAZENECA (FIOCRUZ),219VCD270W,2,2021-10-19,DRIVE THRU PARQUE DE EXPOSIÇÃO,2021,10,19,2ª DOSE
2,3,FEMININO,TRABALHADORES DA SAÚDE,3 - COMIRNATY (PFIZER),32010BD,3,2021-10-07,CENTRO DE VACINAÇÃO UNINASSAU,2021,10,7,REFORÇO
3,4,FEMININO,TRABALHADORES DA SAÚDE,1 - CORONAVAC - SINOVAC (BUTANTAN),210016,1,2021-02-11,CENTRO DE VACINAÇÃO COMPAZ MIGUEL ARRAES,2021,2,11,1ª DOSE
4,5,FEMININO,TRABALHADORES DA SAÚDE,1 - CORONAVAC - SINOVAC (BUTANTAN),210043,2,2021-03-04,CENTRO DE VACINAÇÃO PARQUE DE EXPOSIÇÃO,2021,3,4,2ª DOSE


## Exploração Inicial dos Dados

In [ ]:
# Verificar as primeiras linhas dos dados
df.head()

# Resumo estatístico e informações gerais
df.info()

# Checar valores ausentes
df.isnull().sum()

# Estatísticas descritivas
df.describe()


## Análise da Vacinação Mensal

In [26]:
# Adicionar coluna de 'ano_mes' para facilitar a análise por mês
df['ano_mes'] = df['data_vacinacao'].dt.to_period('M')

# Agrupar o número de vacinados por mês
vacinacao_mensal = df.groupby('ano_mes').size().reset_index(name='total_vacinados')
vacinacao_mensal['data'] = vacinacao_mensal['ano_mes'].dt.to_timestamp()

# Criar gráfico de linha de vacinação mensal
fig1 = px.line(
    vacinacao_mensal,
    x='data',
    y='total_vacinados',
    markers=True,
    title='💉 Vacinação Mensal em Recife',
    labels={'total_vacinados': 'Total de Vacinados', 'data': 'Mês/Ano'},
    color_discrete_sequence=['#1f77b4']
)
fig1.update_layout(template='plotly_white', hovermode='x unified')
# função show() não estava rodando então usei o write_html
fig1.write_html("../content/grafico_vacinacao.html", auto_open=True)


## Distribuição por Tipo de Vacina

In [ ]:
# Contar o número de vacinas por tipo
vacina_tipo = df['vacina'].value_counts().reset_index()
vacina_tipo.columns = ['vacina', 'total']

# Criar gráfico de barras para a distribuição de vacinas
fig2 = px.bar(
    vacina_tipo,
    x='vacina',
    y='total',
    text='total',
    title='📊 Distribuição por Tipo de Vacina',
    labels={'vacina': 'Vacina', 'total': 'Número de Doses'},
    color='vacina',
    color_discrete_sequence=px.colors.qualitative.Set2,
    height=650
)
fig2.update_traces(textposition='outside')
fig2.update_layout(template='plotly_white', xaxis_tickangle=-45)
fig2.write_html("../content/grafico_distribuicao_vacina.html", auto_open=True)


## Análise por Sexo

In [25]:
# Passo 11: Contar o número de vacinados por sexo
sexo_dist = df['sexo'].value_counts().reset_index()
sexo_dist.columns = ['sexo', 'total']

# Passo 12: Criar gráfico de pizza para a distribuição por sexo
fig3 = px.pie(
    sexo_dist,
    names='sexo',
    values='total',
    title='👥 Distribuição por Sexo',
    color='sexo',
    color_discrete_map={'MASCULINO':'#1f77b4', 'FEMININO':'#ff7f0e'}
)
fig3.update_traces(textposition='outside', textinfo='percent+label')
fig3.write_html("../content/analise_sexo.html", auto_open=True)


## Análise por Grupo Prioritário

In [ ]:
# Analisar o total de vacinados por grupo prioritário
top_grupos = df['grupo'].value_counts().nlargest(10).reset_index()
top_grupos.columns = ['grupo', 'total']

# Criar gráfico de barras para os 10 grupos prioritários
fig4 = px.bar(
    top_grupos,
    x='grupo',
    y='total',
    text='total',
    title='📊 Top 10 Grupos Prioritários',
    color='grupo',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    height=650
)
fig4.update_traces(textposition='outside')
fig4.update_layout(template='plotly_white', xaxis_tickangle=-45)
fig4.write_html("../content/analise_grupo_prioritario.html", auto_open=True)


## Análise de Vacinados por Local

In [ ]:
# Analisar os locais de vacinação
top_locais = df['local_vacinacao'].value_counts().nlargest(10).reset_index()
top_locais.columns = ['local_vacinacao', 'total']

# Criar gráfico de barras para os 10 locais de vacinação mais frequentes
fig5 = px.bar(
    top_locais,
    x='local_vacinacao',
    y='total',
    text='total',
    title='🏥 Top 10 Locais de Vacinação',
    color='local_vacinacao',
    color_discrete_sequence=px.colors.qualitative.Vivid,
    height=650
)
fig5.update_traces(textposition='outside')
fig5.update_layout(template='plotly_white', xaxis_tickangle=-45)
fig5.write_html("../content/analise_vacinados_local.html", auto_open=True)

## Análise de Previsões Futuros

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar os dados para a regressão
vacinacao_mensal['mes_ordinal'] = np.arange(len(vacinacao_mensal)).reshape(-1, 1)
X = vacinacao_mensal[['mes_ordinal']]
y = vacinacao_mensal['total_vacinados']

# Criar e treinar o modelo de regressão linear
model = LinearRegression()
model.fit(X, y)

# Fazer previsões para os próximos 3 meses
future_months = np.arange(len(vacinacao_mensal), len(vacinacao_mensal) + 3).reshape(-1, 1)
predicoes = model.predict(future_months)
future_dates = pd.date_range(start=vacinacao_mensal['data'].max() + pd.offsets.MonthBegin(1), periods=3, freq='MS')

# Criar DataFrame de previsões
pred_df = pd.DataFrame({'data': future_dates, 'total_vacinados': predicoes})

# Criar gráfico de linha com previsão
fig6 = px.line(
    vacinacao_mensal,
    x='data',
    y='total_vacinados',
    markers=True,
    title='💉 Vacinação Mensal com Previsões Futuras',
    labels={'total_vacinados': 'Total de Vacinados', 'data': 'Mês/Ano'},
    color_discrete_sequence=['#1f77b4']
)
fig6.add_scatter(
    x=pred_df['data'],
    y=pred_df['total_vacinados'],
    mode='lines+markers',
    line=dict(dash='dot', color='red'),
    name='Previsão'
)
fig6.update_layout(template='plotly_white', hovermode='x unified')
fig6.write_html("../content/analise_previsoes_futuras.html", auto_open=True)


d:\RecifeVax\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

